In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (17.76 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|17,753,073|13|17,753,073|GetValue
|PnLExplainReport:data|1|17,753,060|399|17,753,060|GetValue/Calc
|TradingContainer:NPV|11|17,695,872|138|1,608,715|GetValue
|TradingContainer:NPV|11|17,695,734|90|1,608,703|GetValue/Calc
|Portfolio:items|121|17,413,216|1,151|143,910|GetValue
|Portfolio:items|121|17,413,113|828|143,910|GetValue/Calc
|Workbook:items|1,104|16,570,371|10,701|15,009|GetValue
|Workbook:items|1,100|16,559,670|7,970|15,054|GetValue/Calc
|PnLExplainReport:cutoffs|1|9,839,530|15|9,839,530|GetValue
|PnLExplainReport:cutoffs|1|9,839,514|154|9,839,514|GetValue/Calc
|Root:Clocks|548|9,839,267|1,787,533|17,954|Context
|TradeOpenEvent|1,014|3,902,750|3,902,750|3,848|Db.Get
|TradeOpenEvent:ticket|22,286|3,622,692|112,795|162|GetValue
|TradingTicket|1,014|3,509,897|3,509,897|3,461|Db.Get
|Root:End|278|1,099,073|1,022,866|3,953|Context
|Root:Activity MarketData|255|983,561|912,398|3,857|Context
|Root:Amend Portfolio|255|958,935|888,698|3,760|Context
|Root:Start|270|902,723|831,501|3,343|Context
|Root:Start breaks|255|889,971|824,042|3,490|Context
|Portfolio:children|242|805,165|1,553|3,327|GetValue
|Portfolio:children|121|803,612|896|6,641|GetValue/Calc
|Root:Amend Trading|255|782,598|711,468|3,069|Context
|Root:Activity Portfolio|263|771,260|702,421|2,932|Context
|Root:Activity Trading|263|761,277|694,113|2,894|Context
|Root:Amend MarketData|255|759,468|692,204|2,978|Context
|PortfolioUpdateEvent:children|121|344,189|2,489|2,844|GetValue
|TradingBook|102|315,922|315,922|3,097|Db.Get
|ForwardCashflow:NPV|11|141,078|132|12,825|GetValue
|ForwardCashflow:NPV|11|140,946|92|12,813|GetValue/Calc
|Equity:NPV|15|139,437|170|9,295|GetValue
|Equity:NPV|15|139,267|110|9,284|GetValue/Calc
|MarketInterface:spot|15|132,011|166|8,800|GetValue
|MarketInterface:spot|15|131,844|109|8,789|GetValue/Calc
|ExternalRefData:state|15|124,521|139|8,301|GetValue
|ExternalRefData:state|15|124,382|107|8,292|GetValue/Calc
|RefData:state|15|123,905|143|8,260|GetValue
|RefData:state|15|123,761|116|8,250|GetValue/Calc
|_WorkItemEvent:book2|22,286|104,544|98,661|4|GetValue
|_WorkItemEvent:book1|22,286|104,135|104,135|4|GetValue
|TradeOpenEvent:quantity|22,286|93,991|93,991|4|GetValue
|Clock:cutoffs|2,480|87,637|8,954|35|GetValue
|Clock:cutoffs|20|78,749|133|3,937|GetValue/Calc
|Clock:parent|20|77,852|175|3,892|GetValue
|Clock:parent|20|77,676|148|3,883|GetValue/Calc
|_WorkItemEvent:item|11,143|75,720|69,686|6|GetValue
|TradeOpenEvent:action|11,143|68,877|68,877|6|GetValue
|TradeOpenEvent:premium|11,143|67,109|63,150|6|GetValue
|TradeOpenEvent:unitPrice|11,143|56,962|56,962|5|GetValue
|Event:amends|11,340|56,444|56,444|4|GetValue
|TradingBook:clock|2,200|48,073|17,829|21|GetValue
|PortfolioUpdateEvent|12|46,740|46,740|3,895|Db.Get
|RefDataUpdateEvent|9|32,159|32,159|3,573|Db.Get
|TradingPortfolio|10|31,661|31,661|3,166|Db.Get
|TradingBook:clock|1,100|30,243|10,517|27|GetValue/Calc
|Clock|5|16,302|16,302|3,260|Db.Get
|Portfolio:clock|242|8,253|2,004|34|GetValue
|Portfolio:books|231|7,344|1,669|31|GetValue
|MarketInterface:source|15|6,646|130|443|GetValue
|ClockEvent|2|6,575|6,575|3,287|Db.Get
|Equity:refdata|15|6,568|159|437|GetValue
|MarketInterface:source|15|6,515|107|434|GetValue/Calc
|Equity:refdata|15|6,408|112|427|GetValue/Calc
|Portfolio:clock|121|6,248|1,122|51|GetValue/Calc
|Portfolio:books|121|6,039|839|49|GetValue/Calc
|Equity|2|6,033|6,033|3,016|Db.Get
|ClockEvent:parent|8|5,833|79|729|GetValue
|MarketInterface|2|5,707|5,707|2,853|Db.Get
|MarketDataSource|2|5,641|5,641|2,820|Db.Get
|RootClock|1|4,242|4,242|4,242|Db.Get
|ForwardCashflow|1|3,958|3,958|3,958|Db.Get
|MarketDataSource:clock|30|3,453|262|115|GetValue
|MarketDataSource:clock|15|3,191|147|212|GetValue/Calc
|Entity:clock|40|765|329|19|GetValue
|Entity:clock|20|435|189|21|GetValue/Calc
|MarketInterface:sourceName|15|322|131|21|GetValue
|RefDataUpdateEvent:data|58|283|283|4|GetValue
|RootClock:cutoffs|53|259|189|4|GetValue
|MarketInterface:sourceName|15|190|111|12|GetValue/Calc
|Equity:assetName|15|105|105|7|GetValue
|RootClock:cutoffs|1|69|36|69|GetValue/Calc
|RootClock:cosmicAll|1|19|10|19|GetValue
|CosmicAll:dbState|1|14|9|14|GetValue
|PnLExplainReport:valuable|2|10|10|5|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|PnLExplainReport:ts1|2|9|9|4|GetValue
|PnLExplainReport:ts2|2|8|8|4|GetValue
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [5]:
from mand.graph import DependencyManager, setDependencyManager
from mand.core import Event

class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        #self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items')
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)

    def calculated(self, node):
        if not node.isSimplified:
            return True
        ok = True
        for input in node.tweakPoints():
            if input not in node.ctx.tweaks:
                addFootnote(text='context simplification failure', info='%s in %s on %s in %s' % (str(node.key), node.ctx.name, str(input.key), input.ctx.name))
                print
                print 'node:', node
                print 'tweaks:'
                for t in node.ctx.tweaks:
                    print t
                print 'input:', input
                ok = False
        return ok

    def simplify(self, key):
        if key.fullName() == 'RefData:state':
            obj = key.object()
            return (obj.clock().cutoffs,)
        if key.fullName() == 'Portfolio:items':
            obj = key.object()
            obj2 = obj.getObj(_tr.Clock, 'Trading')
            return (obj.clock().cutoffs, obj2.cutoffs)
        #if key.fullName() == 'Workbook:items':
        #    obj = key.object()
        #    return (obj.clock().cutoffs,)
        
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        s = self.simplify(key)
        if s: 
            nodes = [ getNode(bm) for bm in s ]
            values = [ bm() for bm in s ]
            k = [ (n.object(), n.key.fullName(), v) for n, v in zip(nodes, values) ]
            cKey = tuple(k)
            
            if cKey not in self.contexts:
                tweaks = dict( [ (bm, v) for bm, v in zip(s, values)])
                name = 'simp'
                ctx1 = Context(tweaks, name)
                self.contexts[cKey] = ctx1
            ctx1 = self.contexts[cKey]
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
dm = DM1()
setDependencyManager(dm)


In [6]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (12.25 seconds of wall clock time)

|key|value|
|-|-|
|Context|19
|Db.Get|2176
|GetValue|69506
|GetValue/Calc|1431


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|12,248,667|14|12,248,667|GetValue
|PnLExplainReport:data|1|12,248,653|372|12,248,653|GetValue/Calc
|TradingContainer:NPV|11|12,242,557|123|1,112,959|GetValue
|TradingContainer:NPV|11|12,242,434|81|1,112,948|GetValue/Calc
|Portfolio:items|61|12,083,296|614|198,086|GetValue
|Portfolio:items|55|12,083,216|379|219,694|GetValue/Calc
|Workbook:items|502|11,497,979|5,229|22,904|GetValue
|Workbook:items|500|11,492,750|3,812|22,985|GetValue/Calc
|PnLExplainReport:cutoffs|1|9,754,480|15|9,754,480|GetValue
|PnLExplainReport:cutoffs|1|9,754,464|134|9,754,464|GetValue/Calc
|Root:Clocks|59|9,754,236|37,102|165,326|Context
|Root:Clocks:simp|494|9,679,765|1,919,111|19,594|Context
|TradeOpenEvent|1,014|4,028,806|4,028,806|3,973|Db.Get
|TradeOpenEvent:ticket|10,130|3,209,622|56,415|316|GetValue
|TradingTicket|1,014|3,153,206|3,153,206|3,109|Db.Get
|Root:Activity Portfolio|18|877,712|776|48,761|Context
|Root:Activity Portfolio:simp|244|876,597|800,340|3,592|Context
|Root:Amend Portfolio|11|789,727|570|71,793|Context
|Root:Amend Portfolio:simp|244|788,939|719,551|3,233|Context
|Root:Amend Trading|11|778,343|476|70,758|Context
|Root:Amend Trading:simp|244|777,672|709,194|3,187|Context
|Portfolio:children|110|561,805|828|5,107|GetValue
|Portfolio:children|55|560,976|444|10,199|GetValue/Calc
|PortfolioUpdateEvent:children|55|327,497|1,604|5,954|GetValue
|TradingBook|102|300,554|300,554|2,946|Db.Get
|Equity:NPV|15|98,984|159|6,598|GetValue
|Equity:NPV|15|98,824|129|6,588|GetValue/Calc
|MarketInterface:spot|15|91,497|133|6,099|GetValue
|MarketInterface:spot|15|91,363|98|6,090|GetValue/Calc
|Clock:cutoffs|1,263|88,110|4,775|69|GetValue
|ExternalRefData:state|15|83,837|135|5,589|GetValue
|ExternalRefData:state|15|83,701|95|5,580|GetValue/Calc
|Clock:cutoffs|20|83,412|160|4,170|GetValue/Calc
|Clock:parent|20|82,204|202|4,110|GetValue
|Clock:parent|20|82,002|152|4,100|GetValue/Calc
|_WorkItemEvent:book2|10,130|51,177|45,550|5|GetValue
|_WorkItemEvent:book1|10,130|48,780|48,780|4|GetValue
|TradeOpenEvent:quantity|10,130|43,838|43,838|4|GetValue
|PortfolioUpdateEvent|12|43,668|43,668|3,639|Db.Get
|RefData:state|15|43,648|82|2,909|GetValue
|RefData:state|4|43,565|31|10,891|GetValue/Calc
|_WorkItemEvent:item|5,065|39,723|34,062|7|GetValue
|TradeOpenEvent:premium|5,065|32,958|29,887|6|GetValue
|TradeOpenEvent:action|5,065|32,363|32,363|6|GetValue
|TradingPortfolio|10|30,965|30,965|3,096|Db.Get
|RefDataUpdateEvent|9|29,392|29,392|3,265|Db.Get
|Event:amends|5,148|28,288|28,288|5|GetValue
|TradeOpenEvent:unitPrice|5,065|26,444|26,444|5|GetValue
|Root:End|33|21,782|20,933|660|Context
|TradingBook:clock|1,000|21,692|8,824|21|GetValue
|Clock|5|17,751|17,751|3,550|Db.Get
|Root:Start|26|16,219|15,689|623|Context
|TradingBook:clock|500|12,867|5,000|25|GetValue/Calc
|ClockEvent|2|6,819|6,819|3,409|Db.Get
|MarketInterface:source|15|6,687|128|445|GetValue
|Equity:refdata|15|6,584|160|438|GetValue
|MarketInterface:source|15|6,558|107|437|GetValue/Calc
|Equity:refdata|15|6,423|105|428|GetValue/Calc
|ClockEvent:parent|8|6,303|76|787|GetValue
|Portfolio:clock|171|5,979|1,194|34|GetValue
|MarketInterface|2|5,738|5,738|2,869|Db.Get
|MarketDataSource|2|5,686|5,686|2,843|Db.Get
|Equity|2|5,660|5,660|2,830|Db.Get
|MarketDataSource:clock|23|4,981|194|216|GetValue
|MarketDataSource:clock|19|4,786|139|251|GetValue/Calc
|Portfolio:clock|66|4,784|496|72|GetValue/Calc
|Root:Amend MarketData|11|4,578|615|416|Context
|Root:Amend MarketData:simp|2|3,737|3,654|1,868|Context
|Portfolio:books|105|3,468|796|33|GetValue
|RootClock|1|3,255|3,255|3,255|Db.Get
|ForwardCashflow|1|3,070|3,070|3,070|Db.Get
|Portfolio:books|55|2,844|391|51|GetValue/Calc
|Root:Activity Trading|18|996|686|55|Context
|Entity:clock|40|837|365|20|GetValue
|Root:Start breaks|11|595|420|54|Context
|Root:Activity MarketData|11|575|403|52|Context
|Entity:clock|20|472|196|23|GetValue/Calc
|MarketInterface:sourceName|15|306|130|20|GetValue
|ForwardCashflow:NPV|11|303|118|27|GetValue
|RootClock:cutoffs|53|303|229|5|GetValue
|ForwardCashflow:NPV|11|184|73|16|GetValue/Calc
|MarketInterface:sourceName|15|175|104|11|GetValue/Calc
|Equity:assetName|15|105|105|7|GetValue
|RootClock:cutoffs|1|73|40|73|GetValue/Calc
|RefDataUpdateEvent:data|15|72|72|4|GetValue
|RootClock:cosmicAll|1|19|10|19|GetValue
|CosmicAll:dbState|1|14|10|14|GetValue
|PnLExplainReport:valuable|2|10|10|5|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|PnLExplainReport:ts2|2|8|8|4|GetValue
|PnLExplainReport:ts1|2|8|8|4|GetValue
|CosmicAll:dbState|1|4|4|4|GetValue/Calc

In [7]:
for k, ctx in dm.contexts.items():
    print
    for t in k:
        print t
    for t in ctx.tweaks:
        print '    ', t


(<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-03-28T15:03:25.447049,v=2017-03-28T15:03:25.447047>)
(<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-28T15:03:25.447049,v=2017-03-28T15:03:25.447047>)
     <Node: /Global/Clock/Portfolio.(Clock:cutoffs)() in Root:Clocks:simp @4615300304 T=True>
     <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:Clocks:simp @4615300688 T=True>

(<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-03-28T15:03:25.502909,v=2017-03-28T15:03:25.502907>)
(<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-03-28T15:03:25.502909,v=2017-03-28T15:03:25.447047>)
     <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:Activity Portfolio:simp @4894971920 T=True>
     <Node: /Global/Clock/Portfolio.(Clock:cutoffs)() in Root:Activity Portfolio:simp @4894971728 T=True>

(<Entity:/Global/Clock/MarketData>, 'Clock:cutoffs', <TS:t=2017-03-28T15:03:25.447049,v=2017-03-28T15:03:25.447047>)
     <Node: /Global/Cloc